# Single Machine Total Weighted Tardiness with Precedence

In the single machine total weighted tardiness with precedence ($1|\text{prec}|\sum w_i T_i$), we are given a set of jobs $N = \{ 0, ..., n-1 \}$.
Each job $i \in N$ has the processing time $p_i$, the due date $d_i$, and the weight $w_i$.
We schedule all jobs sequentially in a single machine.
To schedule job $i$, all of its predecessors $P_i \subseteq N$ must be scheduled before.
When job $i$ completes at time $C_i$, its tardiness is $T_i = \max\{ C_i - d_i, 0 \}$.
We want to minimize the total weighted tardiness, $\sum_{i \in N} w_i T_i$.

## DP Formulation

Consider scheduling jobs one by one.
Let $S$ be the set of already scheduled jobs.
We can schedule job $i \notin S$ if $P_i \subseteq S$.
If we schedule $i$ next, its completion time is $C_i = \sum_{j \in S} p_j + p_i$.
Let $V(S)$ be the minimum total weighted tardiness to schedule all jobs in $N \setminus S$.
We have the following DP model:

$$
\begin{align}
    \text{compute } & V(\emptyset) \\
    & V(S) = \begin{cases}
        \min\limits_{i \in N \setminus S : P_i \subseteq S} w_i \max\left\{ \sum\limits_{j \in S} p_j + p_i - d_i, 0 \right\} + V(S \cup \{ i \}) & \text{if } S \neq N \\
        0 & \text{if } S = N
    \end{cases} \\
    & V(S) \geq 0.
\end{align}
$$

## Install DIDPPy

In [1]:
!pip install didppy

## Data

In [2]:
# Number of jobs
n = 5
# Processing times
p = [2, 3, 5, 1, 4]
# Dead dates
d = [1, 4, 7, 2, 5]
# Weights
w = [2, 1, 2, 3, 2]
# Predecessors
capital_p = [[2], [0, 3], [], [], [1]]

## Modeling

In [3]:
import didppy as dp

model = dp.Model()

job = model.add_object_type(number=n)

# S
scheduled = model.add_set_var(object_type=job, target=[])

# N
all_jobs = model.create_set_const(object_type=job, value=list(range(n)))

processing_time = model.add_int_table(p)
predecessors = model.add_set_table(capital_p, object_type=job)

for i in range(n):
    tardiness = dp.max(0, processing_time[scheduled] + processing_time[i] - d[i])
    schedule = dp.Transition(
        name="schedule {}".format(i),
        cost=w[i] * tardiness + dp.IntExpr.state_cost(),
        effects=[(scheduled, scheduled.add(i))],
        preconditions=[~scheduled.contains(i), predecessors[i].issubset(scheduled)],
    )
    model.add_transition(schedule)
    
model.add_base_case([scheduled == all_jobs])

model.add_dual_bound(0)

## Solving

In [4]:
solver = dp.CABS(model, quiet=True)
solution = solver.search()

print("Transitions to apply:")
print("")

for t in solution.transitions:
    print(t.name)

print("")
print("Cost: {}".format(solution.cost))

Transitions to apply:

schedule 3
schedule 2
schedule 0
schedule 1
schedule 4

Cost: 41
